In [1]:
%pip install python-metar

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from metar import Metar
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
bimtra_df = pd.read_csv('data/bimtra_df.csv')
esperas_df = pd.read_csv('data/esperas_df.csv')
metaf_df = pd.read_csv('data/metaf_df.csv')
metar_df = pd.read_csv('data/metar_df.csv')
tc_prev_df = pd.read_csv('data/tc-prev_df.csv')
tc_real_df = pd.read_csv('data/tc-real_df.csv')
test_data = pd.read_csv('data/idsc_test.csv')

In [ ]:
bimtra_df.dt_arr = bimtra_df.dt_arr.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
bimtra_df.dt_dep = bimtra_df.dt_dep.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
esperas_df.hora = esperas_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
metaf_df.hora = metaf_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
metar_df.hora = metar_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
tc_prev_df.hora = tc_prev_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))
tc_real_df.hora = tc_real_df.hora.apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)/1000))

In [ ]:
test_data.head()

In [ ]:
bimtra_df['solution'] = ((bimtra_df.dt_arr - bimtra_df.dt_dep).dt.total_seconds()).astype(int)
bimtra_df.head()

In [ ]:
display(test_data.head())

display(bimtra_df.head())
display(esperas_df.head())
# display(f'metaf_df: {metaf_df.columns}')
# display(f'metar_df: {metar_df.columns}')
display(tc_prev_df.head())
display(tc_real_df.head())

In [ ]:
bimtra_df['hora_ref'] = bimtra_df.dt_dep.apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
bimtra_df.head()

In [ ]:
display(f'idsc_test_df: {test_data.columns}')

display(f'bimtra_df: {bimtra_df.columns}')
display(f'esperas_df: {esperas_df.columns}')
# display(f'metaf_df: {metaf_df.columns}')
# display(f'metar_df: {metar_df.columns}')
display(f'tc_prev_df: {tc_prev_df.columns}')
display(f'tc_real_df: {tc_real_df.columns}')

In [ ]:
print(f'bimtra_df: {bimtra_df.shape}')
print(f'esperas_df: {esperas_df.shape}')
print(f'metaf_df: {metaf_df.shape}')
print(f'metar_df: {metar_df.shape}')
print(f'tc_prev_df: {tc_prev_df.shape}')
print(f'tc_real_df: {tc_real_df.shape}')

In [ ]:
tc_prev_df['aero_tcp'] = tc_prev_df['aero']
tc_prev_df = tc_prev_df.drop(['aero'], axis=1)
tc_real_df['aero_tcr'] = tc_real_df['aero']
tc_real_df = tc_real_df.drop(['aero'], axis=1)

display(tc_prev_df.head())
display(tc_real_df.head())
display(tc_real_df.shape)

In [ ]:
tc_real_df.isna().sum()

In [ ]:
esperas_df['aero_esperas'] = esperas_df['aero']
esperas_df = esperas_df.drop(['aero'], axis=1)
esperas_df['hora_esperas'] = esperas_df['hora']
esperas_df = esperas_df.drop(['hora'], axis=1)
esperas_df['hora_ref'] = esperas_df['hora_esperas']
esperas_df['destino'] = esperas_df['aero_esperas']

esperas_df.head()

In [ ]:
print(f'bimtra_df: {bimtra_df.shape}')
print(f'esperas_df: {esperas_df.shape}')
print(f'metaf_df: {metaf_df.shape}')
print(f'metar_df: {metar_df.shape}')
print(f'tc_prev_df: {tc_prev_df.shape}')
print(f'tc_real_df: {tc_real_df.shape}')

In [ ]:
tc_real_df['hora_ref'] = tc_real_df.hora.apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
tc_prev_df['hora_ref'] = tc_prev_df.hora

tc_real_df['hora_tcr'] = tc_real_df.hora
tc_prev_df['hora_tcp'] = tc_prev_df.hora
tc_real_df = tc_real_df.drop(['hora'], axis=1)
tc_prev_df = tc_prev_df.drop(['hora'], axis=1)

tc_real_df['destino'] = tc_real_df.aero_tcr
tc_prev_df['destino'] = tc_prev_df.aero_tcp


display(tc_real_df.head())
display(tc_prev_df.head())

In [ ]:
tc_prev_df['destino'] = 'SB' + tc_prev_df['destino'].astype(str)
tc_prev_df['destino'].unique()

tc_real_df['destino'] = 'SB' + tc_real_df['destino'].astype(str)
tc_real_df['destino'].unique()

tc_prev_df['aero_tcp'] = 'SB' + tc_prev_df['aero_tcp'].astype(str)
tc_prev_df['aero_tcp'].unique()

tc_real_df['aero_tcr'] = 'SB' + tc_real_df['aero_tcr'].astype(str)
tc_real_df['aero_tcr'].unique()

In [ ]:
train_data = pd.merge(bimtra_df, esperas_df, how='left', on=['hora_ref', 'destino'])
train_data = pd.merge(train_data, tc_prev_df, how='left', on=['hora_ref', 'destino'])
train_data = pd.merge(train_data, tc_real_df, how='left', on=['hora_ref', 'destino'])
# train_data['hora_ref'] = train_data['hora_ref'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
display(train_data.sample(10))
display(train_data.shape)


In [ ]:
train_data.isna().sum() 

In [ ]:
train_data.shape

In [ ]:
train_data = train_data.drop(columns=['nova_cabeceira', 'antiga_cabeceira', 'dt_arr', 'hora_ref', 'hora_esperas', 'dt_dep', 'hora_tcp', 'hora_tcr'], axis=1)

train_data = train_data[['flightid', 'origem', 'destino', 'esperas', 'aero_esperas', 'troca', 'aero_tcp', 'aero_tcr', 'solution']]

train_data.head()

In [ ]:
test_data = test_data.drop(columns=['snapshot_radar', 'path', 'hora_metaf', 'metaf', 'aero_metaf', 'hora_metar', 'metar', 'aero_metar', 'hora_ref', 'hora_esperas', 'dt_dep', 'hora_tcp', 'hora_tcr'], axis=1)
test_data = test_data[['flightid', 'origem', 'destino', 'esperas', 'aero_esperas', 'troca', 'aero_tcp', 'aero_tcr']]
test_data.head()

In [ ]:
cat_columns = ['origem', 'destino', 'aero_esperas', 'aero_tcp', 'aero_tcr']
train_data, test_data = pd.get_dummies(train_data, columns=cat_columns, dtype=int), pd.get_dummies(test_data, columns=cat_columns, dtype=int)

for column in train_data.columns:
    if column == 'solution':
        continue
    if column not in test_data.columns:
        test_data[column] = 0

In [ ]:
train_data.head()

In [ ]:
train_data.describe().columns


In [ ]:
features = train_data.columns
features = features.drop(['flightid', 'solution'])
target = ['solution']

X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data[target], test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Linear Regression MSE: {mean_squared_error(y_test, y_pred)}')

model2 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
y_pred = np.round_(y_pred)
print(y_pred)
print(f'Random Forest Regressor MSE: {mean_squared_error(y_test, y_pred)}')

In [ ]:
X_train, y_train = train_data[features], train_data[target]
X_test = test_data[features]
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = np.round_(y_pred)
print(y_pred)

output = pd.DataFrame({'ID': test_data.flightid, 'solution': y_pred})

output.to_csv('data/submission.csv', index=False)

In [4]:
metar_df.tail()

,hora,metar,aero
106739,1684004400000,METAR SBSV 131900Z 16009KT 9999 BKN020 FEW021TCU 29/24 Q1013=,SBSV
106740,1684008000000,METAR SBSV 132000Z 13007KT 9999 BKN020 29/24 Q1013=,SBSV
106741,1684011600000,METAR SBSV 132100Z 12005KT 9999 SCT020 FEW025TCU 28/24 Q1014=,SBSV
106742,1684015200000,METAR SBSV 132200Z 11006KT 9999 SCT020 FEW025TCU 28/24 Q1014=,SBSV
106743,1684018800000,METAR SBSV 132300Z 11009KT 9999 SCT020 FEW025TCU 28/25 Q1015=,SBSV


In [5]:
# metar_df.metar = metar_df.metar.apply(lambda x: x.replace('=', ' '))
# metar_sample = metar_df['metar'].sample(1).iloc[0]
# print(metar_sample)

# #metar_sample = 'METAR SBRF 152000Z 13005KT 9999 FEW020 SCT070 28/21 Q1011'

# report = Metar.Metar(metar_sample)
# print(report, '\n')

# print(f'Station_Id: {report.station_id}')
# print(f'Type: {report.type}')
# print(f'Time: {report.time}')
# print(f'Day: {report._day}')
# print(f'Temperature: {report.temp}')
# print(f'Dew Point: {report.dewpt}')
# print(f'Wind Direction: {report.wind_dir}')
# print(f'Visibility: {report.vis}')
# print(f'Wind Speed: {report.wind_speed}')
# print(f'Wind Gust: {report.wind_gust}')
# print(f'Pressure: {report.press}')
# print(f'Precipitation: {report.precip_1hr}')


METAR SBRJ 100800Z 31005KT 9000 SCT013 22/21 Q1009 
station: SBRJ
type: routine report, cycle 8 (automatic report)
time: Sun Sep 10 08:00:00 2023
temperature: 22.0 C
dew point: 21.0 C
wind: NW at 5 knots
visibility: 9000 meters
pressure: 1009.0 mb
sky: scattered clouds at 1300 feet
METAR: METAR SBRJ 100800Z 31005KT 9000 SCT013 22/21 Q1009  

Station_Id: SBRJ
Type: METAR
Time: 2023-09-10 08:00:00
Day: 10
Temperature: 22.0 C
Dew Point: 21.0 C
Wind Direction: 310 degrees
Visibility: 9000 meters
Wind Speed: 5 knots
Wind Gust: None
Pressure: 1009.0 mb
Precipitation: None


In [12]:
metar_df.metar = metar_df.metar.apply(lambda x: x.replace('=', ''))
for report in metar_df['metar']:
    try:
        report = Metar.Metar(report)
        metar_df['temperature'] = report.temp.value()
        metar_df['dew_point'] = report.dewpt.value()
        metar_df['wind_direction'] = report.wind_dir
        metar_df['visibility'] = report.vis.value()
        metar_df['wind_speed'] = report.wind_speed.value()
        metar_df['wind_gust'] = report.wind_gust
        metar_df['pressure'] = report.press.value()
        metar_df['precipitation'] = report.precip_1hr
    except:
        continue
    

metar_df.head()

,hora,metar,aero,temperature,dew_point,wind_direction,visibility,wind_speed,wind_gust,pressure,precipitation
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None


In [13]:
metar_df.shape

(106744, 11)

In [14]:
metar_df.head(100)

,hora,metar,aero,temperature,dew_point,wind_direction,visibility,wind_speed,wind_gust,pressure,precipitation
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
5,1654059600000,METAR SBBR 010500Z 27004KT CAVOK 13/08 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
6,1654063200000,METAR SBBR 010600Z 25003KT CAVOK 14/08 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
7,1654066800000,METAR SBBR 010700Z 28002KT CAVOK 12/08 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
8,1654070400000,METAR SBBR 010800Z VRB01KT CAVOK 11/08 Q1018,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None
9,1654074000000,METAR SBBR 010900Z 26002KT CAVOK 12/09 Q1019,SBBR,28.0,25.0,110 degrees,10000.0,9.0,None,1015.0,None


In [17]:
metar_df['wind_direction'].unique()

array([<metar.Datatypes.direction object at 0x287e7db90>], dtype=object)